___
# Ciência dos Dados - Projeto 3: Filmes#
___

### _Gabriela Caruso e Mariana Abrantes - 2C_ ###
___

## Objetivo ##

Desenvolver um programa que cria uma playlist de filmes similares a um outro usado como parâmetro.

___

In [1]:
%matplotlib inline
%reset -f
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
#from emoji import UNICODE_EMOJI
from sklearn import cluster

In [2]:
#df = pd.read_csv('https://query.data.world/s/dgju3rv2knocz2ost6qyly7vwyekcv')
#writer = pd.ExcelWriter('Projeto3_CD.xlsx')
#df.to_excel(writer,'data')
#writer.save()

In [3]:
df = pd.read_excel('Projeto3_CD.xlsx')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
data = df.drop(labels=None, axis=0, index=None, columns='movie_imdb_link', level=None, inplace=False, errors='raise')
data

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [5]:
list(data)

['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

In [6]:
lista_nova = []
for color in data["color"]:
    if color == " Black and White":
        lista_nova.append(1)
    elif color == "Color":
        lista_nova.append(0)
    else:
        lista_nova.append(color)
        
data["color"] = lista_nova
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,0.0,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,0.0,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,0.0,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,0.0,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [7]:
def numerar(dataframe, coluna):
    # Criando a lista de nomes únicos:
    unico = []
    for classe in dataframe[coluna]:
        if classe not in unico:
            unico.append(classe)
            
    # Criando a lista de nomes alterados para números:
    reclassificado = []        
    for item in dataframe[coluna]:
        reclassificado.append(unico.index(item))
    
    # Substituindo a coluna toda do DataFrame:
    dataframe[coluna] = reclassificado
    
    # Retornando o novo DataFrame:
    return dataframe

In [8]:
numerar(data, "actor_2_name")
numerar(data, "director_name")
numerar(data, "actor_1_name")
numerar(data, "movie_title")
numerar(data, "genres")
numerar(data, "actor_3_name")
numerar(data, "language")
numerar(data, "plot_keywords")
numerar(data, "country")
numerar(data, "content_rating")
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,0.0,0,723.0,178.0,0.0,855.0,0,1000.0,760505847.0,0,...,3054.0,0,0,0,237000000.0,2009.0,936.0,7.9,1.78,33000
1,0.0,1,302.0,169.0,563.0,1000.0,1,40000.0,309404152.0,1,...,1238.0,0,0,0,300000000.0,2007.0,5000.0,7.1,2.35,0
2,0.0,2,602.0,148.0,0.0,161.0,2,11000.0,200074175.0,2,...,994.0,0,1,0,245000000.0,2015.0,393.0,6.8,2.35,85000
3,0.0,3,813.0,164.0,22000.0,23000.0,3,27000.0,448130642.0,3,...,2701.0,0,0,0,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,4,NaN,NaN,131.0,NaN,4,131.0,NaN,4,...,NaN,1,2,1,NaN,NaN,12.0,7.1,NaN,0


In [9]:
data.corr()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
color,1.000000,0.076877,-0.019863,0.041400,0.041791,-0.031764,0.116301,-0.037117,-0.043514,0.077772,...,0.044222,0.026895,0.014427,0.181217,-0.021039,-0.308490,-0.033302,0.145357,-0.048963,-0.031396
director_name,0.076877,1.000000,-0.402579,-0.235617,-0.171347,-0.191387,0.670401,-0.173570,-0.454951,0.286195,...,-0.378396,0.156405,0.173626,0.231178,-0.109031,-0.045655,-0.217259,-0.150929,-0.059965,-0.219607
num_critic_for_reviews,-0.019863,-0.402579,1.000000,0.258486,0.180674,0.271646,-0.378324,0.190016,0.480601,-0.190457,...,0.609387,-0.054838,-0.076676,-0.112067,0.119994,0.275707,0.282306,0.305303,-0.049786,0.683176
duration,0.041400,-0.235617,0.258486,1.000000,0.173296,0.123558,-0.227497,0.088449,0.250298,-0.090839,...,0.328403,0.047433,0.035259,-0.051661,0.074276,-0.135038,0.131673,0.261662,-0.090071,0.196605
director_facebook_likes,0.041791,-0.171347,0.180674,0.173296,1.000000,0.120199,-0.113314,0.090723,0.144945,-0.051268,...,0.221890,-0.032447,-0.053382,-0.012438,0.021090,-0.063820,0.119601,0.170802,0.001642,0.162048
actor_3_facebook_likes,-0.031764,-0.191387,0.271646,0.123558,0.120199,1.000000,-0.252846,0.249927,0.308026,-0.125243,...,0.230189,-0.065521,-0.075804,-0.105054,0.047451,0.096137,0.559662,0.052633,-0.003366,0.278844
actor_2_name,0.116301,0.670401,-0.378324,-0.227497,-0.113314,-0.252846,1.000000,-0.235565,-0.400148,0.300860,...,-0.320273,0.205744,0.190224,0.284554,-0.099611,-0.177983,-0.315503,-0.038040,-0.011548,-0.218339
actor_1_facebook_likes,-0.037117,-0.173570,0.190016,0.088449,0.090723,0.249927,-0.235565,1.000000,0.154468,-0.117311,...,0.145461,-0.073448,-0.080008,-0.084993,0.022639,0.086873,0.390487,0.076099,-0.020049,0.135348
gross,-0.043514,-0.454951,0.480601,0.250298,0.144945,0.308026,-0.400148,0.154468,1.000000,-0.268205,...,0.559958,-0.116625,-0.154476,-0.227066,0.102179,0.030886,0.262768,0.198021,0.069346,0.378082
genres,0.077772,0.286195,-0.190457,-0.090839,-0.051268,-0.125243,0.300860,-0.117311,-0.268205,1.000000,...,-0.184317,0.048480,0.048948,0.164911,-0.034089,-0.139400,-0.127125,-0.016946,-0.016681,-0.122175


In [10]:
#k_means = cluster.KMeans(n_clusters=3)
#k_means.fit(data)